# AI HW2: Cutting Stocks Problem


In [1]:
import math
import random
import numpy as np

In [2]:
def generate_permutation(array):
    '''This function generates a permutation of the given array'''
    permutation = np.copy(array)
    return np.random.permutation(permutation)

In [3]:
def cost_function(array, length):
    '''this function calculates number of rolls that 
    are used to fulfill the input requests'''
    remained = length
    total_cost = 1
    for element in array:
        if element <= remained:
            remained -= element
        else:
            remained = length - element
            total_cost += 1
    return total_cost

In [4]:
def find_neighbor_HC(requests):
    '''This function finds neighbors for the current 
    permutation which is used in hill climbing algorithm'''
    neighbor = np.copy(requests)
    for i in range(2):
        ind1, ind2 = np.random.randint(len(requests) - 1), np.random.randint(len(requests) - 1)
        neighbor[ind1], neighbor[ind2] = neighbor[ind2], neighbor[ind1]
    return neighbor   


### Hill Climbing Algorithm

In [5]:
def cutting_stock_Hill_Climbing_solver(length, requests, max_counter_limit):
    '''This function tries to solve the cutting stock problem with HC'''
    current_request = generate_permutation(requests)

    current_cost    = cost_function(current_request, length)
    min_cost = current_cost
    counter = 0

    while True:
        neighbor      = find_neighbor_HC(current_request)
        neighbor_cost = cost_function(neighbor, length)

        if neighbor_cost <= current_cost:
            current_request = neighbor
            current_cost    = neighbor_cost
            if neighbor_cost == current_cost:
                counter += 1
            else:
                counter = 0
        else:
            counter += 1
        
        if current_cost < min_cost:
            min_cost = current_cost
        if counter > max_counter_limit:
            return min_cost
    # return min_cost

In [6]:
def driver_HC(file_path, limit):
    with open(file_path, 'r') as test_case:
        file_lines  = test_case.readlines()
        roll_length = int(file_lines[0].split()[2])
        requests    = np.array(list(map(int, file_lines[3].split(', ' ))))
        res = cutting_stock_Hill_Climbing_solver(roll_length, requests, limit)
        return res

In [10]:
for i in range(4):
    file_p = 'input%i.stock' % (i+1)
    print("test case%i: " %(i+1))
    print("limit: %i" %(50000), "rolls: ", driver_HC(file_p,5000))
    print("limit: %i" %(1000), "rolls: ", driver_HC(file_p,1000))
    print("limit: %i" %(5000), "rolls: ", driver_HC(file_p,5000))
    print("limit: %i" %(20000), "rolls: ", driver_HC(file_p,20000))


test case1: 
limit: 50000 rolls:  53
limit: 1000 rolls:  54
limit: 5000 rolls:  54
limit: 20000 rolls:  52
test case2: 
limit: 50000 rolls:  80
limit: 1000 rolls:  83
limit: 5000 rolls:  80
limit: 20000 rolls:  79
test case3: 
limit: 50000 rolls:  99
limit: 1000 rolls:  103
limit: 5000 rolls:  99
limit: 20000 rolls:  97
test case4: 
limit: 50000 rolls:  224
limit: 1000 rolls:  228
limit: 5000 rolls:  224
limit: 20000 rolls:  221


### Simulated Anealing 

In [11]:
def find_neighbor_SA(requests):
    '''this function finds neighbors for the current permutation
    in SA algorithm'''
    number_of_requests = len(requests)
    neighbor = np.copy(requests)
    ind1, ind2 = np.random.randint(number_of_requests - 1), np.random.randint(number_of_requests - 1)
    neighbor[ind1], neighbor[ind2] = neighbor[ind2], neighbor[ind1]
    return neighbor 

In [12]:
def acceptance_probability(current_cost, neighbor_cost, temperature):
    '''this function calculates the accpetance probability which
    is used in the SA algorithm'''
    return math.exp((current_cost-neighbor_cost)/temperature)

In [13]:
def cutting_stock_SA_solver(requests, length, min_temperature, max_temperature, alpha, number_of_iterations):
    '''This function tries to solve the cutting stock problem
    with SA algorithm'''
    current_permutation = requests.copy()
    current_cost = cost_function(current_permutation, length)
    optimal_cost = current_cost
    optimal_permutation = np.copy(current_permutation)
    temperature = max_temperature
    while temperature > min_temperature:
        for iteration in range(number_of_iterations):
            neighbor = find_neighbor_SA(current_permutation)
            neighbor_cost = cost_function(neighbor, length)
            probability = acceptance_probability(current_cost, neighbor_cost, temperature)
            if neighbor_cost < current_cost or probability > np.random.rand():
                current_permutation = neighbor
                current_cost = neighbor_cost
            if current_cost < optimal_cost:
                optimal_cost = current_cost
                optimal_permutation = current_permutation
        temperature *= alpha

    return optimal_cost

In [14]:
def driver_SA(file_path, min_temp, max_temp, alpha, iterations):
    with open(file_path, 'r') as test_case:
            file_lines  = test_case.readlines()
            roll_length = int(file_lines[0].split()[2])
            requests    = np.array(list(map(int, file_lines[3].split(', ' ))))
            res = cutting_stock_SA_solver(requests, roll_length, min_temp, max_temp, alpha, iterations)
            return res

In [15]:
for i in range(4):
    file_p = 'input%i.stock' % (i+1)
    print("test case %i: " % (i+1))
    print("min temp: %f, max temp: %f, alpha: %f, iterations: %i" %(0.01, 1, 0.99, 1000), "rolls: ",  driver_SA(file_p, 0.01, 1, 0.99, 1000))
    print("min temp: %f, max temp: %f, alpha: %f, iterations: %i" %(0.01, 1, 0.99, 100),  "rolls: ", driver_SA(file_p, 0.01, 1, 0.99, 100))
    print("min temp: %f, max temp: %f, alpha: %f, iterations: %i" %(0.01, 1, 0.5, 1000),  "rolls: ", driver_SA(file_p, 0.01, 1, 0.5, 1000))
    print("min temp: %f, max temp: %f, alpha: %f, iterations: %i" %(0.01, 10, 0.99, 1000),  "rolls: ", driver_SA(file_p, 0.01, 10, 0.99, 1000))
    print("min temp: %f, max temp: %f, alpha: %f, iterations: %i" %(0.01, 1, 0.99, 2000),  "rolls: ", driver_SA(file_p, 0.01, 1, 0.99, 2000))


test case 1: 
min temp: 0.010000, max temp: 1.000000, alpha: 0.990000, iterations: 1000 rolls:  51
min temp: 0.010000, max temp: 1.000000, alpha: 0.990000, iterations: 100 rolls:  52
min temp: 0.010000, max temp: 1.000000, alpha: 0.500000, iterations: 1000 rolls:  52
min temp: 0.010000, max temp: 10.000000, alpha: 0.990000, iterations: 1000 rolls:  51
min temp: 0.010000, max temp: 1.000000, alpha: 0.990000, iterations: 2000 rolls:  51
test case 2: 
min temp: 0.010000, max temp: 1.000000, alpha: 0.990000, iterations: 1000 rolls:  78
min temp: 0.010000, max temp: 1.000000, alpha: 0.990000, iterations: 100 rolls:  80
min temp: 0.010000, max temp: 1.000000, alpha: 0.500000, iterations: 1000 rolls:  81
min temp: 0.010000, max temp: 10.000000, alpha: 0.990000, iterations: 1000 rolls:  78
min temp: 0.010000, max temp: 1.000000, alpha: 0.990000, iterations: 2000 rolls:  78
test case 3: 
min temp: 0.010000, max temp: 1.000000, alpha: 0.990000, iterations: 1000 rolls:  96
min temp: 0.010000, max

## Genetic

In [16]:
population_size = 100

In [17]:
def generate_population(chromosome):
    '''this function creates a generation with generation size length
    using the given chromosome'''
    chromosome_copy = np.copy(chromosome)
    population = np.array([])
    for _ in range(population_size):
        indivisual = np.random.permutation(chromosome_copy)
        population = np.append(population, indivisual)
        population = population.reshape(-1, len(indivisual))
    return population

In [18]:
def fitness(chromosome, length):
    '''this function calculates the fitness of the given permutation
    which is the number of rolls that are use. so& th less the fitness, the better the result'''
    remained = length
    fitness_score = 1
    for gene in chromosome:
        if gene <= remained:
            remained -= gene
        else:
            remained = length - gene
            fitness_score += 1
    
    return fitness_score

In [20]:
def evaluate_population(population, length):
    '''this function calculates the fitness for evey indivisual in the
    population'''
    population_fitness = np.array([])
    for indivisual in population:
        population_fitness = np.append(population_fitness, fitness(indivisual, length))
    return population_fitness
    

In [21]:
def tournament_selection(population, population_fitness, tournament_size=3):
    '''this function tries to find the best random parents for the new generation'''
    size = population[0].size
    parents = []
    parents_indices = []
    for i in range(population_size):
        candidates = random.sample(range(0, population_size), tournament_size)
        best_fit     = -1
        parent_index = -1
        for c in candidates:
            if population_fitness[c] < best_fit:
                best_fit = population_fitness[c]
                parent_index = c
        parents.append(population[parent_index])
        parents_indices.append(parent_index)
    parents = np.array(parents)
    parents_indices = np.array(parents_indices)
    return parents, parents_indices

In [22]:
def crossover(population, parents, dictionary, requests, crossover_rate):
    '''this function tries to do the crossover with probability crossover rate'''
    children = []
    cnt = 0
    parents_copy = np.copy(parents)
    parents_copy = list(parents_copy)
    size = population[0].size

    while cnt < population_size:
        parent1, parent2 = parents_copy[cnt], parents_copy[cnt+1]
        child1, child2 = [], []

        cross_point = np.random.randint(1, size-2)

        for i in range(cross_point+1):
            child1.append(parent1[i])
            child2.append(parent2[i])
        
        child1_dict = {}
        child2_dict = {}

        for key in dictionary:
            child1_dict[key] = 0
            child2_dict[key] = 0

        for gene in child1:
            child1_dict[gene] += 1
        for gene in child2:
            child2_dict[gene] += 1

        choice = np.random.rand()

        if choice < crossover_rate:
                
            ind1, ind2 = cross_point+1, cross_point+1
            while len(child1) < len(parent1):
                if dictionary[parent2[ind1]] - child1_dict[parent2[ind1]] > 0:
                    child1_dict[parent2[ind1]] += 1
                    child1.append(parent2[ind1])
                ind1 = (1+ind1) % size

            while len(child2) < len(parent1):
                if dictionary[parent1[ind2]] - child2_dict[parent1[ind2]] > 0:
                    child2_dict[parent1[ind2]] += 1
                    child2.append(parent1[ind2])
                ind2 = (1+ind2) % size
        else:
            child1 = parent1[:]
            child2 = parent2[:]
        
        children.append(child1)
        children.append(child2)
        cnt += 2
    return np.array(children)

In [23]:
def generate_new_generation(population, parents_fitness, children, children_fitness, combine_rate):
    '''this functiom tries to make the new generation based on the previous
    population and the new children that are created'''
    new_generation = []
    population_copy = np.copy(population)
    population_copy = list(population_copy)
    for indivisual in population_copy:
        new_generation.append(indivisual)
    

    for i in range(int(combine_rate * population_size)):

        max_child = -1
        ind1 = -1
        for j in range(population_size):
            if children_fitness[i] < max_child:
                max_child = children_fitness[i]
                ind1 = i
        children_fitness[ind1] = -1

        min_parent = 100
        ind2 = -1
        for j in range(population_size):
            if parents_fitness[i] > min_parent:
                min_parent = parents_fitness[i]
                ind2 = i
        parents_fitness[ind2] = 100

        new_generation[ind2] = children[ind1]
    
    return new_generation

In [24]:
def mutation(children, mutation_rate):
    '''this function tries to do the mutation in the children'''
    for child in children:
        rate = np.random.rand()
        if rate < mutation_rate:
            ind1, ind2 = np.random.randint(len(child)), np.random.randint(len(child))
            child[ind1], child[ind2] = child[ind2], child[ind1]
    return children

In [25]:
def cutting_stock_GA_solver(length, chromosome, crossover_rate, mutation_rate, combine_rate, generation_length):
    '''this function tries to solve the cutting stocks problem with genetic algorithm'''
    population = generate_population(chromosome)
    population_fitness = evaluate_population(population, length)
    generation = 0
    best_fit = np.inf
    while generation < generation_length:
        parents, parents_indices_in_population = tournament_selection(population, population_fitness)
        parents_indices_in_population = np.array(parents_indices_in_population, dtype=int)

        parents_fitness = np.array([])
        for i in parents_indices_in_population:
            parents_fitness = np.append(parents_fitness, population_fitness[i])
        
        dictionary = {}
        for key in chromosome:
            if key not in dictionary:
                dictionary[key] = 1
            else:
                dictionary[key] += 1

        children = crossover(population, parents, dictionary, chromosome, crossover_rate)
        children = mutation(children, mutation_rate)
        children_fitness = evaluate_population(children, length)
        new_population = generate_new_generation(population, parents_fitness, children, children_fitness, combine_rate)
        new_population_fitness = evaluate_population(new_population, length)
        if min(new_population_fitness) < best_fit:
            best_fit = int(min(new_population_fitness))
        population = np.array(new_population)

        generation += 1
    return best_fit

In [26]:
def driver_GA(file_path, crossover_rate, mutation_rate, combine_rate, generation_length):
    with open(file_path, 'r') as test_case:
        file_lines  = test_case.readlines()
        roll_length = int(file_lines[0].split()[2])
        requests    = np.array(list(map(int, file_lines[3].split(', ' ))), dtype=int)
        res = cutting_stock_GA_solver(roll_length, requests, crossover_rate, mutation_rate, combine_rate, generation_length)
        return res

In [27]:
for i in range(4):
    file_p = 'input%i.stock' % (i+1)
    print("test case%i: " % (i+1))
    print("crossover rate: %f, mutation rate: %f, combination rate: %f" %(0.5, 0.5, 0.3), driver_GA(file_p, 0.5, 0.5, .3, 2000))
    print("crossover rate: %f, mutation rate: %f, combination rate: %f" %(0.5, 0.8, 0.3), driver_GA(file_p, 0.5, 0.5, .3, 2000))
    print("crossover rate: %f, mutation rate: %f, combination rate: %f" %(0.5, 0.5, 0.8), driver_GA(file_p, 0.5, 0.5, .3, 2000))
    print("crossover rate: %f, mutation rate: %f, combination rate: %f" %(0.8, 0.5, 0.3), driver_GA(file_p, 0.5, 0.5, .3, 2000))
    print("crossover rate: %f, mutation rate: %f, combination rate: %f" %(0.8, 0.5, 0.4), driver_GA(file_p, 0.5, 0.5, .3, 2000))
    print("crossover rate: %f, mutation rate: %f, combination rate: %f" %(0.8, 0.5, 0.4), driver_GA(file_p, 0.5, 0.5, .3, 5000))

test case1: 
crossover rate: 0.500000, mutation rate: 0.500000, combination rate: 0.300000, #generations: 2000 rolls:  58
crossover rate: 0.500000, mutation rate: 0.800000, combination rate: 0.300000, #generations: 2000 rolls:  57
crossover rate: 0.500000, mutation rate: 0.500000, combination rate: 0.800000, #generations: 2000 rolls:  58
crossover rate: 0.800000, mutation rate: 0.500000, combination rate: 0.300000, #generations: 2000 rolls:  56
crossover rate: 0.800000, mutation rate: 0.500000, combination rate: 0.400000, #generations: 2000 rolls:  54
crossover rate: 0.800000, mutation rate: 0.500000, combination rate: 0.400000, #generations: 5000 rolls:  52
test case2: 
crossover rate: 0.500000, mutation rate: 0.500000, combination rate: 0.300000, #generations: 2000 rolls:  85
crossover rate: 0.500000, mutation rate: 0.800000, combination rate: 0.300000, #generations: 2000 rolls:  86
crossover rate: 0.500000, mutation rate: 0.500000, combination rate: 0.800000, #generations: 2000 roll